# Exercise

Data given in the problem

In [1]:
# Coap message sizes (in bits)
Lgetreq = 60 * 8
Lgetresp = 55 * 8

# MQTT message sizes (in bits)
Lcon = 54 * 8
Lconack = 47 * 8
Lsub = 58 * 8
Lsuback = 52 * 8
Lpub = 68 * 8

# Energy costs (in millijoule or millijoule/bit)
Etx = 50 * 1e-6
Erx = 58 * 1e-6
Ec = 2.4

#### EQ1) Compute the total energy consumed by the two battery-powered devices over a period of 24 hours in both cases when using COAP (a) and MQTT (b), using each in its most efficient configuration energy-wise.

EQ1a) For CoAP we identified the observation mode as the most efficient configuration energy-wise, where the temperature sensor is theh CoAP server and the valve is the client. In this way the valve does only one GET request with the observe flag set and the temperature sensor sends a GET response every 5 minutes. We assumed the temperature value changes every 5 minutes or that we could program it to do so, and also that given the ideal connection we could use non confirmable messages without worrying.

<img src="./images/COAP.png" alt="COAP" width="300"/>

In [2]:
Evalvecoap = Etx*Lgetreq + 48 * (6 * Erx*Lgetresp + Ec)
Etempcoap = Erx*Lgetreq + 48 * 6 * Etx*Lgetresp
Ecoap = Evalvecoap + Etempcoap

print(f"Total energy consumed by valve and temperature sensor is: {Ecoap:.2f} mJ")

Total energy consumed by valve and temperature sensor is: 128.94 mJ


EQ1b) For MQTT we use the RPi as the broker and for the same hyphotesis from before (connection is ideal) we used QoS 0. Both the temperature sensor and the valve connect to the broker, the valve subscribes to the topic and the temperature sensor publish the temperature every five minutes.

<img src="./images/MQTT.png" alt="MQTT" width="300"/>

In [3]:
Evalvemqtt = Etx*Lcon + Erx*Lconack + Etx*Lsub + Erx*Lsuback + 48 * (6 * Erx*Lpub + Ec)
Etempmqtt = Etx*Lcon + Erx*Lconack + 48 * 6 * Etx*Lpub
Emqtt = Evalvemqtt + Etempmqtt

print(f"Total energy consumed by valve and temperature sensor is: {Emqtt:.2f} mJ")

Total energy consumed by valve and temperature sensor is: 132.25 mJ


#### EQ2) Propose atleast one solution for decreasing the energy consumption when passing using the Raspberry PI as a broker. Give a rough estimate of the energy saving that could be obtained with your solution: recompute the energy under your proposed configuration.

EQ2-1) At first we tought of lowering message sizes. Using MQTT-SN it is true that we have to add a step in the beginning, the registration phase, but all publish messages will be smaller. We got the messages dimensions for Reg and RegAck from the last exercise seen in class (all the other sizes corresponded so we think it's a good estimate) and removed 8 byte (10 byte previous topic - 2 byte for the topicID) from the publish messages, we know it's not exactly like this but it's to give an estimate. We also tought of reducing the payload size since 8 bytes are too much for a temperature value, 4 bytes are sufficient.

In [4]:
Lreg = 59 * 8
Lregack = 51 * 8
LpubSN = Lpub - (8 * 8) - (4 * 8)

Evalvemqtt = Etx*Lcon + Erx*Lconack + Etx*Lsub + Erx*Lsuback + 48 * (6 * Erx*LpubSN + Ec)
Etempmqtt = Etx*Lcon + Erx*Lconack + Etx*Lreg + Erx*Lregack + 48 * 6 * Etx*LpubSN
Emqtt = Evalvemqtt + Etempmqtt

print(f"Total energy consumed by valve and temperature sensor is: {Emqtt:.2f} mJ")

Total energy consumed by valve and temperature sensor is: 129.32 mJ


EQ2-2) Looking online we found that MQTT-SN also has a QoS -1 (also called 3) that permits to a client to publish a message without connecting or registering (fire and forget). It's a small improvement but we tought it was relevant. We just removed connection and registration from the temperature sensor size.

In [5]:
Evalvemqtt = Etx*Lcon + Erx*Lconack + Etx*Lsub + Erx*Lsuback + 48 * (6 * Erx*LpubSN + Ec)
Etempmqtt = 48 * 6 * Etx*LpubSN
Emqtt = Evalvemqtt + Etempmqtt

print(f"Total energy consumed by valve and temperature sensor is: {Emqtt:.2f} mJ")

Total energy consumed by valve and temperature sensor is: 129.23 mJ


EQ2-3) The most energy expensive operations is the average of the temperatures so if we could the best thing is to move it on the RPi. We could do this even without modifying the broker by adding a client subscribed on the temperature that calculates the average and just publish the update to the valve (see image below). This is a massive improvement even using normal MQTT (as we did) but maybe we modified the system to much.

<img src="./images/BetterMQTT.png" alt="BetterMQTT" width="400"/>

In [6]:
# Do computation on the Raspberry Pi and share only the average every 30 min with the valve
Evalvemqtt = Etx*Lcon + Erx*Lconack + Etx*Lsub + Erx*Lsuback + 48 * Erx*Lpub
Etempmqtt = Etx*Lcon + Erx*Lconack + 48 * 6 * Etx*Lpub
Emqtt = Evalvemqtt + Etempmqtt

print(f"Total energy consumed by valve and temperature sensor is: {Emqtt:.2f} mJ")

Total energy consumed by valve and temperature sensor is: 9.48 mJ
